In [1]:
import os
import pandas as pd
import xlrd
from pathlib import Path
import openpyxl
from openpyxl import load_workbook
import numpy as np
import re
import ast
from xls2xlsx import XLS2XLSX

In [2]:

    ##Getting the 1045 data
xlsx_file = Path('SimData', '/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster/Alvarado Hospital/106370652_CDM_All_2020.xlsx')
wb_obj = openpyxl.load_workbook(xlsx_file) 

In [3]:
#Function to extract 1045 sheet name from xlsx file only

def find1045_xlsx(filepath):
    #read file
    xlsx_file = Path(filepath)
    wb_obj = openpyxl.load_workbook(xlsx_file)

    #get sheet names
    sheetnames=wb_obj.sheetnames

    #Create a keywords list to identify ABS 1045 forms
    keywords= ["common", "procedures", "1045", "form"]

    #Search for the sheet by keyword
    for name in sheetnames:
        wholename=name.lower()
        wordsInName=wholename.split()
        match =  any(item in wordsInName for item in keywords)
        if match==True:
            sheetname=(wb_obj[name])
            return(sheetname)
            break


In [4]:
#Function to extract all dataframes from a single xlsx file
def getdata_xlsx(file):
    dflist=[]
    coordlist=[]
    sheet=find1045_xlsx(file)
    alphabet=["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z"]
    found=False
    keywords1 = ["evaluation", "management", "services"]
    keywords2 = ["laboratory", "pathology", "services"]
    keywords3= ["radiology", "services"]
    keywords4 = ["medicine", "services"]
    keywords5 = ["surgery", "services"]
    keywords6 = ["other", "common", "outpatient", "procedures"]
    keywords7 = ["count", "of", "reported", "procedures",]
    keywordslists=[keywords1,keywords2,keywords3,keywords4,keywords5,keywords6,keywords7]
    for wordlist in keywordslists:
        for row in sheet.iter_rows():
                for cell in row:
                    heading = str(cell.value).lower()
                    wordsInHeading=heading.split()
                    match =  all(item in wordsInHeading for item in wordlist)
                    if match==True:
                        coord1=cell.coordinate
                        coordlist.append(coord1)
                        break
    for coordinate in coordlist[0:-1]:
        splitcoord=re.split('(\d+)', coordinate)
        nextbreak=coordlist[int(coordlist.index(coordinate)+1)]
        splitbreak=re.split('(\d+)', nextbreak)
        lastcell=f"{int(splitbreak[1])-1}"
        coord2=f"C{lastcell}"

        cellgroup=[coordinate,coord2]
        df=pd.DataFrame(sheet[f"{cellgroup[0]}:{cellgroup[1]}"])



        dflist.append(df)

    return(dflist)

  
    

In [5]:
#Function to convert cell object values to plaintext values
def convertvalue(x):
    return x.value

In [6]:
#Convert all dataframes from a single sheet into plaintext
sheet=find1045_xlsx(xlsx_file)
masterlist=[]
rawlist=getdata_xlsx(xlsx_file)
for i in range(len(rawlist)):
    df=rawlist[i].applymap(convertvalue)
            #add the hospital name as a variable
    masterlist.append(df)

In [7]:
#Get the column names from the dataframe
for df in masterlist:
    if len(df)>0:
        df.columns = df.iloc[0]
        df["Service Type"]=df.iloc[0,0]
        df.rename(columns={ df.columns[0]: "Service" }, inplace = True)
        df=df.drop(df.index[0], inplace=True)




In [8]:
#add the hospital name as a variable in every df in the converted list
for i in range(len(masterlist)):
        raw_name=sheet["A1"].value
        hos_name=raw_name.replace('Hospital Name: ', '')
        masterlist[i]["Hospital"]=hos_name


In [9]:
masterlist[0]

,Service,2019 CPT Code,Average Charge,Service Type,Hospital
1,"Emergency Room Visit, Level 2 (low to moderate...",99282,746.55,Evaluation & Management Services (CPT Codes 99...,Alvarado Hospital
2,"Emergency Room Visit, Level 3 (moderate severity)",99283,1051.05,Evaluation & Management Services (CPT Codes 99...,Alvarado Hospital
3,"Emergency Room Visit, Level 4 (high severity)",99284,1581.3,Evaluation & Management Services (CPT Codes 99...,Alvarado Hospital
4,"Outpatient Visit, established patient, 15 minutes",99213,None,Evaluation & Management Services (CPT Codes 99...,Alvarado Hospital


In [10]:
#get the names of all folders in the data directory
os.chdir("..")
os.chdir("data")
os.chdir("2020_CDM_chargemaster")



In [11]:
#Function to extract 1045 sheet name

def find1045(filepath):
    #read file
    if filepath[-5:] == ".xlsx":
        excel_file = Path(filepath)
        wb_obj = openpyxl.load_workbook(excel_file)

    #get sheet names
    sheetnames=wb_obj.sheetnames

    #Create a keywords list to identify ABS 1045 forms
    keywords= ["common", "procedures", "1045", "form"]

    #Search for the sheet by keyword
    for name in sheetnames:
        wholename=name.lower()
        wordsInName=wholename.split()
        match =  any(item in wordsInName for item in keywords)
        if match==True:
            sheetname=(wb_obj[name])
            return(sheetname)
            break


In [12]:
chargemaster=os.getcwd()

In [13]:
os.chdir("..")

In [14]:
os.chdir(chargemaster)

In [15]:
#DO NOT RUN
#get the names of all files from each directory, then test if each file has a 1045 in it.
dirlist = list(os. listdir())
for i in range(len(dirlist)):
    os.chdir(dirlist[i])
    for root, dirs, files in os.walk("."):
        for filename in files:
            try:
                print("found one")
                print(find1045_xlsx(filename))
                print(filename)
            except:
                print("wrong file type")
                print(filename)
    os.chdir(chargemaster)

found one
<Worksheet "Common OP Procedures">
106390923_CDM_All_2020.xlsx
found one
<Worksheet "AB 1045 Form">
106104023_CDM_All_2020.xls.xlsx
found one
None
106374465_Common25_2020.xlsx
found one
None
106370730_CDM_2020.xlsx
found one
None
106370730_PCT_CHG_2020.xlsx
found one
None
106370730_Common25_2020.xlsx
found one
wrong file type
Thumbs.db
found one
<Worksheet "AB 1045 Form">
106361339_CDM_All_2020.xlsx
found one
wrong file type
Thumbs.db
found one
wrong file type
106361339_Comments_2020.docx
found one
None
106314024_PCT_CHG_2020.xls.xlsx
found one
None
106314024_CDM_2020.xlsx
found one
None
106314024_PCT_CHG_2020-Cloud.xls.xlsx
found one
None
106314024_Common25_2020.xlsx
found one
wrong file type
Thumbs.db
found one
<Worksheet "AB 1045 Form CMH">
106560473_CDM_All_2020.xlsx
found one
<Worksheet "AB 1045 Form">
106190197_Common25_2020.xlsx
found one
wrong file type
106190197_Comments_2020.docx
found one
None
106190197_CDM_2020.xlsx
found one
<Worksheet "AB 1045 Form">
106331293_C

In [ ]:
filename="test.xlsx"

In [ ]:
filename[-5:]

In [ ]:
os.chdir("data")

In [ ]:
os.getcwd()

In [ ]:
os.chdir("..")

In [ ]:
os.chdir("2020_CDM_chargemaster")

In [ ]:

    ##Getting the 1045 data
xls_file = Path("/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster/Emanate Health Foothill Presbyterian Hospital/106190298_Common25_2020.xls")

In [ ]:
book = xlrd.open_workbook(xls_file)

In [ ]:
book = xlrd.open_workbook(xls_file)

print("The number of worksheets is {0}".format(book.nsheets))
print("Worksheet name(s): {0}".format(book.sheet_names()))
sh = book.sheet_by_index(0)
print("{0} {1} {2}".format(sh.name, sh.nrows, sh.ncols))
print("Cell D30 is {0}".format(sh.cell_value(rowx=29, colx=3)))
for rx in range(sh.nrows):
    print(sh.row(rx))

In [ ]:
#Function to extract 1045 sheet from xls file (not name, whole sheet)

def find1045_xls(filepath):
    #read file
    xls_file = Path(filepath)
    book = xlrd.open_workbook(xls_file)

    #get sheet names
    sheetnames=("{0}".format(book.sheet_names()))
    sheetnames= ast.literal_eval(sheetnames)


#Create a keywords list to identify ABS 1045 forms
    keywords= ["common", "procedures", "1045", "form"]

#Search for the sheet by keyword
    for name in sheetnames:
        wholename=name.lower()
        wordsInName=wholename.split()
        match =  any(item in wordsInName for item in keywords)
        if match==True:
            cur_sheet = book.sheet_by_name(name)
            return(cur_sheet)
            break


In [ ]:
x2x = XLS2XLSX(xls_file)


In [ ]:
from xls2xlsx import XLS2XLSX
x2x = XLS2XLSX(xls_file)
wb = x2x.to_xlsx()

In [ ]:
884

>>> import ast
>>> x = '[ "A","B","C" , " D"]'
>>> x = ast.literal_eval(x)
>>> x
['A', 'B', 'C', ' D']
>>> x = [n.strip() for n in x]
>>> x
['A', 'B', 'C', 'D']

In [ ]:

import xlrd
from openpyxl.workbook import Workbook as openpyxlWorkbook

# content is a string containing the file. For example the result of an http.request(url).
# You can also use a filepath by calling "xlrd.open_workbook(filepath)".

xlsBook = xlrd.open_workbook(file_contents=content)
workbook = openpyxlWorkbook()

for i in xrange(0, xlsBook.nsheets):
    xlsSheet = xlsBook.sheet_by_index(i)
    sheet = workbook.active if i == 0 else workbook.create_sheet()
    sheet.title = xlsSheet.name

    for row in xrange(0, xlsSheet.nrows):
        for col in xrange(0, xlsSheet.ncols):
            sheet.cell(row=row + 1, column=col + 1).value = xlsSheet.cell_value(row, col)

# The new xlsx file is in "workbook", without iterators (iter_rows).
# For iteration, use "for row in worksheet.rows:".
# For range iteration, use "for row in worksheet.range("{}:{}".format(startCell, endCell)):".

In [ ]:
#Iterate through files and convert all xls files to xlsx, removing the original file
def convert(filename):
    p.save_book_as(file_name=f"{filename}",
                dest_file_name=f"{filename[:-4]}.xlsx" )

In [ ]:
import pyexcel as p
data=p.get_records(file_name=xls_file)
p.save_as(array=data, dest_file_name="example.xls")

In [ ]:
os.getcwd()

In [63]:
def cleanse_func(v):
     v = v.replace("&nbsp;", "")
     v = v.rstrip().strip()
     return v


In [60]:
from pyexcel_xls import get_data
data = get_data(xls_file)


In [54]:
cwd=os.getcwd()

In [9]:
os.getcwd()

'/home/ygael/Projects/healthcare/code'

In [10]:
os.chdir("..")

In [53]:
cwd

'/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster'

In [11]:
os.chdir("data")

In [12]:
os.chdir("2020_CDM_chargemaster")

In [ ]:
#A function to convert all files in subfolders starting from the absolute filepath of the year's chargemaster folder

def convertfrom(directory):
    #Use absolute path to move to the directory containing all hospital folders
    os.chdir(directory)
    #save current working directory to a variable
    cwd=os.getcwd()
    #Convert all files to xlsx format
    dirlist = list(os. listdir())
    for i in range(len(dirlist)):
        os.chdir(dirlist[i])
        for root, dirs, files in os.walk("."):
            for filename in files:
                if filename[-3:]=="xls":
                    try:
                        convert(filename)
                        os.remove(filename)
                    except:
                        print(filename)
                else:
                    continue
        os.chdir("..")
    os.chdir(cwd)

In [21]:
convertfrom("/home/ygael/Projects/healthcare/data/2020_CDM_chargemaster")

106364188_CDM_2020.xls
106364188_CDM_2020-Cloud.xls
106332172_CDM_2020.xls
106332172_CDM_2020-Cloud.xls
106201281_PCT_CHG_2020.xls


data=pd.read_excel(xls_file)
data = data.applymap(lambda x: x.encode('unicode_escape').
                 decode('utf-8') if isinstance(x, str) else x)
data.to_excel

In [16]:
book = xlrd.open_workbook(xls_file)


#character = table.Cell(Row = 1, Column = 1).Range.Text
#character = removeSpecialCharacters(character)

In [15]:
os.chdir("../data/sqlite")

In [16]:
from sqlalchemy import create_engine

In [27]:
engine = create_engine('sqlite:///hospitaldata.db', echo=True)
sqlite_connection = engine.connect()

2021-05-26 15:45:28,085 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-05-26 15:45:28,086 INFO sqlalchemy.engine.base.Engine ()
2021-05-26 15:45:28,089 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-05-26 15:45:28,091 INFO sqlalchemy.engine.base.Engine ()


In [28]:
sqlite_table = "masterlist0"
masterlist[0].to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2021-05-26 15:45:29,099 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("masterlist0")
2021-05-26 15:45:29,101 INFO sqlalchemy.engine.base.Engine ()
2021-05-26 15:45:29,104 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("masterlist0")
2021-05-26 15:45:29,105 INFO sqlalchemy.engine.base.Engine ()
2021-05-26 15:45:29,112 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE masterlist0 (
	"index" BIGINT, 
	"Service" TEXT, 
	"2019 CPT Code" BIGINT, 
	"Average Charge" FLOAT, 
	"Service Type" TEXT, 
	"Hospital" TEXT
)


2021-05-26 15:45:29,113 INFO sqlalchemy.engine.base.Engine ()
2021-05-26 15:45:29,123 INFO sqlalchemy.engine.base.Engine COMMIT
2021-05-26 15:45:29,124 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_masterlist0_index ON masterlist0 ("index")
2021-05-26 15:45:29,125 INFO sqlalchemy.engine.base.Engine ()
2021-05-26 15:45:29,136 INFO sqlalchemy.engine.base.Engine COMMIT
2021-05-26 15:45:29,142 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-05-26 15

In [31]:
sqlite_connection.close()

In [32]:
os.system('sqlite3')

0